In [1]:
!pip install datasets transformers[sentencepiece] sacrebleu -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 17.4 MB/s eta 0:00:00


In [2]:
import os
import sys
import transformers
import tensorflow as tf
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import AdamWeightDecay
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM

In [3]:
model_checkpoint = "thilina/mt5-sinhalese-english"

In [4]:
raw_datasets = load_dataset("zaanind/sinhala_englsih_parrel_corpus")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/80684 [00:00<?, ? examples/s]

In [5]:
# Split the dataset into training, validation, and test sets
train_test_split = raw_datasets['train'].train_test_split(test_size=0.30, seed=42)
train_dataset = train_test_split['train']
test_val_split = train_test_split['test'].train_test_split(test_size=0.5, seed=42)
validation_dataset = test_val_split['train']
test_dataset = test_val_split['test']

# Confirm the sizes of each set
print(f"Training set size: {len(train_dataset)}")
print(f"Validation set size: {len(validation_dataset)}")
print(f"Test set size: {len(test_dataset)}")

Training set size: 56478
Validation set size: 12103
Test set size: 12103


In [6]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['english', 'sinhala'],
        num_rows: 80684
    })
})

In [7]:
train_dataset

Dataset({
    features: ['english', 'sinhala'],
    num_rows: 56478
})

In [8]:
validation_dataset

Dataset({
    features: ['english', 'sinhala'],
    num_rows: 12103
})

In [9]:
test_dataset

Dataset({
    features: ['english', 'sinhala'],
    num_rows: 12103
})

In [10]:
train_dataset[1]

{'english': "i'm confused sir", 'sinhala': 'මම අවුල් වෙනවා සර්'}

In [11]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/393 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/644 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [12]:
tokenizer("Hello, this is a sentence!")

{'input_ids': [30273, 261, 714, 339, 259, 262, 259, 98923, 309, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [13]:
tokenizer(["Hello, this is a sentence!", "This is another sentence."])

{'input_ids': [[30273, 261, 714, 339, 259, 262, 259, 98923, 309, 1], [1494, 339, 259, 7845, 259, 98923, 260, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1]]}

In [14]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["ලියාපදිංචිය සම්පූර්ණ කලාට පස්සෙ ඔයාට තහවුරු කිරීමේ කේතයක්  හම්බවේවි"]))

{'input_ids': [[50043, 146489, 12430, 115838, 28433, 75079, 8242, 68077, 64845, 1489, 259, 45820, 61543, 3165, 76171, 5872, 4887, 2177, 22846, 9071, 1851, 3085, 147217, 5005, 8892, 42963, 10431, 7299, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [15]:
max_input_length = 128
max_target_length = 128

def preprocess_function(examples):
    # Access the English and Sinhala text directly using their keys
    inputs = examples['english']  # This should be a list of English sentences
    targets = examples['sinhala']  # This should be a list of Sinhala sentences

    # Tokenize the inputs (English sentences)
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets (Sinhala sentences)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    # Assign the tokenized IDs to the 'labels' key
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [16]:
preprocess_function(train_dataset[:2])

{'input_ids': [[1532, 2847, 259, 7505, 27416, 1], [259, 266, 277, 282, 259, 148056, 16273, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[259, 62331, 1489, 2168, 9291, 18590, 129232, 14246, 38040, 1], [259, 11600, 8246, 59772, 61359, 259, 119245, 1]]}

In [17]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/80684 [00:00<?, ? examples/s]

In [18]:
tokenized_datasets1 = train_dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/56478 [00:00<?, ? examples/s]

In [19]:
tokenized_datasets2 = validation_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/12103 [00:00<?, ? examples/s]

In [20]:
tokenized_datasets3 = test_dataset .map(preprocess_function, batched=True)

Map:   0%|          | 0/12103 [00:00<?, ? examples/s]

In [21]:
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

tf_model.h5:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/tf_keras/src/initializers/initializers.py:121: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(
All model checkpoint layers were used when initializing TFMT5ForConditionalGeneration.

All the layers of TFMT5ForConditionalGeneration were initialized from the model checkpoint at thilina/mt5-sinhalese-english.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMT5ForConditionalGeneration for predictions without further training.


In [22]:
batch_size = 16
learning_rate = 2e-5
weight_decay = 0.01
num_train_epochs = 10

In [23]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

In [24]:
generation_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf", pad_to_multiple_of=128)

In [25]:
train_dataset = model.prepare_tf_dataset(
    tokenized_datasets1,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_collator,
)

In [26]:
validation_dataset = model.prepare_tf_dataset(
    tokenized_datasets2,
    batch_size=batch_size,
    shuffle=False,
    collate_fn=data_collator,
)

In [27]:
generation_dataset = model.prepare_tf_dataset(
    tokenized_datasets3,
    batch_size=8,
    shuffle=False,
    collate_fn=generation_data_collator,
)


In [28]:
optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)
model.compile(optimizer=optimizer)

In [29]:
model.fit(train_dataset, validation_data=validation_dataset, epochs=3)

Epoch 1/3


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
3529/3529 [==============================] - 2327s 617ms/step - loss: 1.4721 - val_loss: 1.1420
Epoch 2/3
3529/3529 [==============================] - 2143s 607ms/step - loss: 1.3036 - val_loss: 1.0796
Epoch 3/3
3529/3529 [==============================] - 2139s 606ms/step - loss: 1.2291 - val_loss: 1.0414


In [30]:
model.save_pretrained("tf_model/")

In [31]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = TFAutoModelForSeq2SeqLM.from_pretrained("tf_model/")

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/tf_keras/src/initializers/initializers.py:121: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(
All model checkpoint layers were used when initializing TFMT5ForConditionalGeneration.

All the layers o

In [32]:
input_text  = "My name Ashan.I am a software engineer"

tokenized = tokenizer([input_text], return_tensors='np')
out = model.generate(**tokenized, max_length=128)
print(out)

tf.Tensor(
[[     0   2880   4512    259  69886   2022  57461   3252    259  11600
    2880  20439  16172  29054  14284   3686   8157  40805  17635   5077
  194080      1]], shape=(1, 22), dtype=int32)


In [33]:
with tokenizer.as_target_tokenizer():
    print(tokenizer.decode(out[0], skip_special_tokens=True))

මගේ නම අෂාන් මම මෘදුකාංග ඉංජිනේරුවෙක්


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [34]:

def translate_sentences(tokenizer, model, sentences):
    # Tokenize the input text with appropriate configuration
    tokenized = tokenizer(sentences, return_tensors='tf', padding=True, truncation=True, max_length=128)

    # Generate translations using the model
    outputs = model.generate(**tokenized, max_length=128)

    # Decode and print each translation
    translated_sentences = []
    for output in outputs:
        translated_text = tokenizer.decode(output, skip_special_tokens=True)
        translated_sentences.append(translated_text)

    return translated_sentences

In [36]:
new_sentences = [
    "This is a book.",
    "What is your name?",
    "I am learning to code.",
    "It is a sunny day today.",
    "She likes to watch movies.",
    "How much does this cost?",
    "Can you help me?",
    "I need a new laptop for work.",
    "Where is the nearest hospital?",
    "What time is it now?",
    "He is my brother.",
    "This is a beautiful garden.",
    "Do you know where she is?",
    "I would like to have some tea.",
    "What should I do now?",
    "He is playing soccer.",
    "Can you bring me my phone?",
    "I am cooking dinner.",
    "They are going to the market.",
    "She will be coming tomorrow."
]

expected_translations = [
    "මේක පොතක්.",
    "ඔබේ නම කුමක්ද?",
    "මම කේතය ඉගෙන ගන්නවා.",
    "අද දවස හිරු එලිය තිබෙනවා.",
    "ඇයට ෆිල්ම් බලන්න කැමතියි.",
    "මේකට කීයද?",
    "ඔයාට මට උදව් කරන්න පුළුවන්ද?",
    "මට වැඩ සඳහා අලුත් ලැප්ටොප් එකක් ඕනේ.",
    "ලඟම රෝහල කොහේද?",
    "දැන් වෙලාව කීයද?",
    "ඔහු මගේ සහෝදරයා.",
    "මේක ලස්සන උද්‍යානයක්.",
    "ඇය කොහේද කියලා ඔයාට දැනෙනවාද?",
    "මට තේ ටිකක් ඕනේ.",
    "දැන් මට මොනවාද කල යුතුද?",
    "ඔහු පාපන්දු ක්‍රීඩා කරනවා.",
    "මට මගේ දුරකතනය ගෙන්නන්න පුළුවන්ද?",
    "මම රෑට කෑම හදනවා.",
    "ඔවුන් වෙළඳසැලට යනවා.",
    "ඇය හෙට එනවා."
]




In [37]:
# Translate the sentences
translated_sentences = translate_sentences(tokenizer, model, new_sentences)

# Print the translated sentences
for original, translated in zip(new_sentences, translated_sentences):
    print(f'Original: {original} \nTranslated: {translated}\n')

Original: This is a book. 
Translated: මේක පොතක්

Original: What is your name? 
Translated: ඔයාගෙ නම මොකක්ද

Original: I am learning to code. 
Translated: මම කේතය ඉගෙන ගන්නවා

Original: It is a sunny day today. 
Translated: අද දවසේ සනීප දවසක්

Original: She likes to watch movies. 
Translated: එයා චිත් රපට බලන්න කැමතියි

Original: How much does this cost? 
Translated: මේකට කීයක් වියදම් කරන්නද

Original: Can you help me? 
Translated: ඔයාට මට උදව් කරන්න පුළුවන්ද

Original: I need a new laptop for work. 
Translated: මට වැඩට අලුත් ලැප්ටොප් එකක් ඕනේ

Original: Where is the nearest hospital? 
Translated: ළඟම ඉස්පිරිතාලයේ කොහෙද තියෙන්නේ

Original: What time is it now? 
Translated: දැන් වෙලාව කීයද

Original: He is my brother. 
Translated: එයා මගේ සහෝදරයා

Original: This is a beautiful garden. 
Translated: මේක ලස්සන උද් යානයක්

Original: Do you know where she is? 
Translated: ඔයා දන්නවාද එයා කොහෙද කියලා

Original: I would like to have some tea. 
Translated: මම තේ එකක් බොන්න කැමතියි

Original: Wh

In [38]:
def calculate_accuracy(expected, generated):
    correct_matches = sum([1 for exp, gen in zip(expected, generated) if exp.strip() == gen.strip()])
    total = len(expected)
    accuracy = correct_matches / total * 100
    return accuracy


Manual test Accuracy

In [40]:
accuracy = calculate_accuracy(expected_translations, translated_sentences)
print(f"Manual test set accuracy: {accuracy:.2f}%")

Manual test set accuracy: 0.00%


Manual test Accuracy is 15.00% because out of 20 there is only 3 sentenses exactly correct.But most of them are same meaning with different words.